#### importing 

In [1]:
%load_ext autoreload
%autoreload 2 
import pandas as pd
import numpy as np
import pdfplumber
import re
import nltk
from urllib.parse import urljoin, quote_plus, quote, urlencode
from urllib.request import Request, urlopen
from io import StringIO, BytesIO
from housecredit import HouseCredit
from try_scrape_demand import DemandDeposit

/Users/olgacravesana/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
abanca={1:{'link':'https://www.abanca.pt', 'page':None}, 
        2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf', 'page':[6,7]},
        3:{'link':'https://storage.googleapis.com/bank_price_pdfs/1_all_products_210412130040.pdf', 'page':6}}

bankinter={1:{'link':'https://www.bankinter.pt', 'page':None},
           2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf', 'page':[8,9,10,11,12,13,14,15,16,17]},
           3:{'link': 'https://storage.googleapis.com/bank_price_pdfs/2_all_products_210412130350.pdf', 'page':None, 'notes':'image file'}}

bic={1:{'link':'https://www.bancobic.ao' ,'page':None},
     2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0079_/0079_PRE_0.pdf', 'page':[7,8,9,10,12,13]},
     3:{'link':'https://storage.googleapis.com/bank_price_pdfs/3_all_products_210412130840.pdf', 'page':None, 'notes':'wrong file'}}

bai={1:{'link':'https://www.bancobai.ao', 'page':None},
     2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0008_/0008_PRE.pdf', 'page':None, 'notes':'No housing credit products'},
     3:{'link':'https://storage.googleapis.com/bank_price_pdfs/4_all_products_210412130952.pdf', 'page':None, 'notes':'No housing credit products'}}

ctt={1:{'link':'https://www.bancoctt.pt/', 'page':None},
     2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0193_/0193_PRE.pdf', 'page':6},
     3:{'link':'https://storage.googleapis.com/bank_price_pdfs/5_all_products_210412131014.pdf', 'page':6}}

#### dictionary

In [4]:
house_credit_com = {'admin':['Comissões associadas a atos administrativos 4.1 Não realização da escritura',
                             'Alteração do local da escritura',
                             'Declarações de dívida',
                             'Mudança de regime de crédito',
                             'Declarações de dívida',
                             'Pedido de 2ª via de Caderneta Predial',
                             'Emissão de declarações não obrigatórias por lei',
                             'Emissão de 2ª vias de Declaração para efeitos de IRS – Urgente',
                             'Emissão de 2º vias de Declaração para efeitos de IRS',
                             'Emissão de 2ª vias de faturas',
                             'Declaração de Dívida para Fins Diversos',
                             'Declaração de Encargos com Prestações'],
                    'certificates':['Emolumentos do registo predial', 'registo predial',
                                    'Certidão permanente on-line'],
                    'debt_recovery':['Comissão de recuperação de valores em dívida', 'Prestação até 50.000 €',
                                    'Prestação > 50.000 €', 'Comissão de recuperação de valores em dívida',
                                    'Prestação > 50.000,00€', 'Prestação ≤ 50.000,00€'],
                    'displacement':['Comissão de deslocação', 'Até 100 Kms', '101 a 250 Kms', '> 250 Kms '],
                    'early_payment':['Comissão de reembolso antecipado parcial', 'Taxa fixa', 'Taxa variável', 
                                    'Taxa fixa', 'Comissão de reembolso antecipado total', 'Comissão de antecipação',
                                    '(pré.aviso 7 dias)', 'Comissão de compra antecipada', '(pré-aviso 10 dias)',
                                    'Comissão de Reembolso Antecipado Parcial',
                                    'Comissão de reembolso antecipado total'],
                    'evaluation':['Avaliação', 'Imóvel residencial', 
                                 'Garagens e arrecadações não anexas ao imóvel residencial', 'Avaliação do Imóvel'],
                    'formalization':['Comissão de formalização', 'Formalização'],
                    'process':['Processo', 'Abertura de Processo',
                              'Desistência ou não conclusão do processo por motivos imputáveis ao cliente'],
                    'inspections':['Vistorias', 'em caso de construção ou realização de obras'],
                    'reanalysis':['Reanálise'],
                    'settlement':['Comissão de Liquidação de Prestação', 'Liquidação de Prestação'],
                    'solicitors_notary':['Emolumentos notariais', 'Solicitadoria', 'Notiário'],
                    'statements':['Emissão de extratos de conta de empréstimos liquidados', 'extrato', 'extratos',
                                  'extrato de conta', 'extrato mensal'],
                    'taxes':['Imposto do Selo sobre concessão de crédito', 'imposto', 'imposto de selo', 'impostos'],
                    'termination':['Cessação da posição contratual', 'cessação', 'rescisão', 'encerramento']}

#### make function of pedro run